In [1]:
import torch.distributions

In [3]:
d = torch.distributions.one_hot_categorical.OneHotCategorical(probs=torch.tensor([.3,.3,.4]))
d.sample(sample_shape=(100,)).shape

torch.Size([100, 3])

In [2]:
import torch
torch.__version__

'2.0.1'

In [2]:
import torch
from diffusers import DiffusionPipeline, UNet2DConditionModel
from mvdiffusion.models.unet_mv2d_condition import UNetMV2DConditionModel

In [4]:




pipeline = DiffusionPipeline.from_pretrained(
'flamehaze1115/wonder3d-v1.0', # or use local checkpoint './ckpts'
custom_pipeline='flamehaze1115/wonder3d-pipeline',
torch_dtype=torch.float16,
cache_dir = './GNN3D/weights'
)

Couldn't connect to the Hub: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/flamehaze1115/wonder3d-v1.0 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc5e8257dc0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: b182ac7e-1f96-44a2-8f64-1cf6aad7a07a)').
Will try to load from local cache.


OSError: Cannot load model flamehaze1115/wonder3d-v1.0: model is not cached locally and an error occured while trying to fetch metadata from the Hub. Please check out the root cause in the stacktrace above.

In [4]:
# try load model id
model_id = 'D:/work/GNN3D/weights/models--lambdalabs--sd-image-variations-diffusers/snapshots/42bc0ee1726b141d49f519a6ea02ccfbf073db2e/unet'
revision = 'v2.0'

unet = UNetMV2DConditionModel.from_pretrained(model_id,in_channels=8,low_cpu_mem_usage=False, ignore_mismatched_sizes=True)


Some weights of UNetMV2DConditionModel were not initialized from the model checkpoint at D:/work/GNN3D/weights/models--lambdalabs--sd-image-variations-diffusers/snapshots/42bc0ee1726b141d49f519a6ea02ccfbf073db2e/unet and are newly initialized because the shapes did not match:
- conv_in.weight: found shape torch.Size([320, 4, 3, 3]) in the checkpoint and torch.Size([320, 8, 3, 3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
wonder_id = 'C:/Users/Xiang/.cache/huggingface/hub/models--flamehaze1115--wonder3d-v1.0/snapshots/d6d2efc033a06a74d3761268de7295c97e6935d2/unet'
unet = UNetMV2DConditionModel.from_pretrained(wonder_id)

The config attributes {'cross_domain_attention': False, 'joint_attention': False, 'joint_attention_twice': True} were passed to UNetMV2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [3]:
unet.enable_xformers_memory_efficient_attention()

In [5]:
unet.config

FrozenDict([('sample_size', 32),
            ('in_channels', 8),
            ('out_channels', 4),
            ('center_input_sample', False),
            ('flip_sin_to_cos', True),
            ('freq_shift', 0),
            ('down_block_types',
             ['CrossAttnDownBlockMV2D',
              'CrossAttnDownBlockMV2D',
              'CrossAttnDownBlockMV2D',
              'DownBlock2D']),
            ('mid_block_type', 'UNetMidBlockMV2DCrossAttn'),
            ('up_block_types',
             ['UpBlock2D',
              'CrossAttnUpBlockMV2D',
              'CrossAttnUpBlockMV2D',
              'CrossAttnUpBlockMV2D']),
            ('only_cross_attention', False),
            ('block_out_channels', [320, 640, 1280, 1280]),
            ('layers_per_block', 2),
            ('downsample_padding', 1),
            ('mid_block_scale_factor', 1),
            ('act_fn', 'silu'),
            ('norm_num_groups', 32),
            ('norm_eps', 1e-05),
            ('cross_attention_dim', 768),
 

In [7]:
# test input, output
# predict the noise residual
# latent_model_input (12,8,32,32)
# image embeddings (12,1,768)
# class_labels 

latent_model_input = torch.randn(size = (12,8,32,32)).to('cuda')
image_embeddings = torch.randn(size = (12,1,768)).to('cuda')
camera_embeddings = torch.randn(size = (12,10)).to('cuda')
t = torch.tensor(950.).to('cuda')
unet.to('cuda')
noise_pred = unet(latent_model_input, t, encoder_hidden_states=image_embeddings, class_labels=camera_embeddings).sample


In [9]:
noise_pred

tensor([[[[ 1.6451, -0.7944,  2.0928,  ...,  1.3754,  0.6563,  1.7017],
          [ 1.1369,  0.0205,  0.3306,  ...,  0.1336, -1.4247, -0.0861],
          [-0.7323, -0.7516,  1.4478,  ..., -0.0486,  0.4949,  1.3067],
          ...,
          [ 0.1738,  0.0539, -0.7371,  ...,  0.7548,  1.1351, -0.1594],
          [-1.1450,  1.0165, -0.0416,  ..., -0.2504,  0.7586,  2.4113],
          [-1.1008,  0.5582,  0.2755,  ...,  0.7030,  2.4381,  0.5441]],

         [[-0.1480, -1.2908,  0.6926,  ..., -1.0000, -0.6202,  0.3504],
          [-3.0567, -0.6264,  1.1325,  ...,  0.6143,  0.2469,  0.2414],
          [ 1.4290, -1.4810, -0.3120,  ...,  0.8861,  1.1703, -0.0359],
          ...,
          [-1.6988,  0.2924, -0.7695,  ...,  0.1713,  0.9562, -1.1710],
          [ 0.3860,  0.4149,  0.9479,  ...,  0.6074,  0.1053, -0.9838],
          [ 0.6721,  1.1418,  1.4959,  ...,  0.5179,  0.5675,  2.6051]],

         [[-0.0511,  0.0584, -0.5652,  ...,  1.7786, -0.2101,  0.5164],
          [-0.7629,  0.1641, -

In [1]:
import torch
import torchvision.transforms as transforms

D:\Apps\Miniconda\envs\MLpy39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\Apps\Miniconda\envs\MLpy39\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
transforms.CenterCrop(144)()

In [1]:
class Dict2Class(object):
    def __init__(self, my_dict):
        for key in my_dict:
            setattr(self, key, my_dict[key])

In [5]:
from data.mvds import MVDataset
from transformers import  CLIPImageProcessor
import yaml
config_path = 'configs/train_config.yml'
wonder_id='C:/Users/Xiang/.cache/huggingface/hub/models--flamehaze1115--wonder3d-v1.0/snapshots/d6d2efc033a06a74d3761268de7295c97e6935d2'
feature_extractor = CLIPImageProcessor.from_pretrained(wonder_id,subfolder='feature_extractor')
with open(config_path,'r') as file:
    cfg = yaml.safe_load(file)
# cfg = yaml.safe_load(args.config_path)
cfg.update({'config_path': config_path})
args = Dict2Class(cfg)
dataset = MVDataset(args,feature_extractor)

In [12]:
list(dataset[0].keys())

['image_cond',
 'image_target',
 'image_cond_vae',
 'image_cond_CLIP',
 'image_target_vae',
 'camera_embed']

In [17]:
dataset[0]['image_cond_vae'].shape

torch.Size([3, 256, 256])

In [1]:
from mvdiffusion.pipelines.pipeline_mvdiffusion_image import MVDiffusionImagePipeline
from mvdiffusion.models.unet_mv2d_condition import UNetMV2DConditionModel
from diffusers import AutoencoderKL, DDIMScheduler
from transformers import CLIPVisionModelWithProjection, CLIPImageProcessor

/pfs/mt-1oY5F7/luoyihao/anaconda3/envs/GNN3D/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/pfs/mt-1oY5F7/luoyihao/anaconda3/envs/GNN3D/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/pfs/mt-1oY5F7/luoyihao/anaconda3/envs/GNN3D/lib/python3.9/site-packages/diffusers/models/dual_transformer_2d.py:20: FutureWarning: `DualTransformer2DModel` is deprecated and will be removed in version 0.29. Importing `DualTransformer2DModel` from `diffusers.models.dual_transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.transformers.dual_transformer_2d import DualTransformer2DMode

In [2]:
model_path = '/pfs/mt-1oY5F7/luoyihao/project/GNN3D/weights/wonder3d'

vae=AutoencoderKL.from_pretrained(model_path, subfolder='vae')
image_encoder = CLIPVisionModelWithProjection.from_pretrained(model_path, subfolder = 'image_encoder')
feature_extractor = CLIPImageProcessor.from_pretrained(model_path, subfolder = 'feature_extractor')
unet = UNetMV2DConditionModel.from_pretrained(model_path, subfolder = 'unet', num_views=2)
scheduler = DDIMScheduler.from_pretrained(model_path, subfolder = 'scheduler')
pipeline = MVDiffusionImagePipeline(vae=vae,image_encoder = image_encoder,unet=unet,scheduler=scheduler,safety_checker=None,
                                    feature_extractor=feature_extractor,requires_safety_checker=False,num_views=2)

The config attributes {'cross_domain_attention': False, 'joint_attention': False, 'joint_attention_twice': True} were passed to UNetMV2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [3]:
pipeline.unet.enable_xformers_memory_efficient_attention()
pipeline = pipeline.to('cuda')


In [5]:
import os
from PIL import Image

imagepath = '/pfs/mt-1oY5F7/luoyihao/project/GNN3D/validation_debug'

def read_images_in_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):  # Check for image file extensions
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert("RGB")
            images.append(image)
    return images

images = read_images_in_folder(imagepath)
out = pipeline(images[1], num_inference_steps=20, output_type='pt', guidance_scale=1, height=320, width=320).images


100%|██████████| 20/20 [00:03<00:00,  5.30it/s]


In [29]:
images[0].size

(320, 320)

In [6]:
from torchvision.utils import make_grid, save_image
result = make_grid(out.cpu(), nrow = 6,padding=0,value_range=(0,1))

In [7]:
save_image(result, 'result4.png')

In [12]:
list1 = [i for i in range(3)]
list2 = [i for i in range(4)]

list1.extend(list2)
print(list1)

import torch
array = torch.randn(size = (12,10))
array[list1].shape

[0, 1, 2, 0, 1, 2, 3]


torch.Size([7, 10])